# **RVC AI Cover Maker (separate parts version)**
Created by [ShiromiyaG](https://github.com/ShiromiyaG)
- Colab inspired on [AICoverGen](https://github.com/SociallyIneptWeeb/AICoverGen) by [SociallyIneptWeeb](https://github.com/SociallyIneptWeeb)
- Uses the [blaise-tk](https://github.com/blaise-tk) version of [RVC_CLI](https://github.com/blaise-tk/RVC_CLI)

In [ ]:
import tarfile
import os
from google.colab import drive
from pathlib import Path

def extract_tar_file(tar_file_path, destination_path):
  if not Path(tar_file_path).exists():
      print(f"Tar file {tar_file_path} does not exist.")
      return

  extraction_failed = False

  with tarfile.open(tar_file_path, "r:gz") as tar:
      for member in tar.getmembers():
          try:
              tar.extract(member, destination_path)
          except Exception as e:
              print(f"Failed to extract file {member.name}: {e}")
              extraction_failed = True

  print(f"Extraction of {tar_file_path} to {destination_path} completed.")

  if os.path.exists(tar_file_path):
      try:
          os.remove(tar_file_path)
      except Exception as e:
          print(f"Failed to remove tar file {tar_file_path}: {e}")

  if extraction_failed:
      print("Extraction encountered errors.")

drive.mount('/content/drive')
Path('/content/dependencies').mkdir(parents=True, exist_ok=True)
os.chdir("/content")
# @title # **Install**
# @markdown ##### Yes, it takes time
voc_extra_files = True
# @markdown ##### To find bugs, you have the option of not clearing the output **(Causes crashes)**
not_clean_outputs = False #@param {type:"boolean"}
# @markdown ##### Install precompiled pip dependencies for faster installation **(May be dated)**
precompiled = True #@param {type:"boolean"}
if voc_extra_files:
  !git clone https://github.com/ShiromiyaG/Music_Source_Separation_Training.git
  !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt -d Music_Source_Separation_Training/models -o model_bs_roformer_ep_317_sdr_12.9755.ckpt
  !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/Music-Source-Separation/model_bs_roformer_ep_317_sdr_12.9755.yaml -d Music_Source_Separation_Training/models -o model_bs_roformer_ep_317_sdr_12.9755.yaml
  !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/MDX23C-8KFFT-InstVoc_HQ.ckpt -d Music_Source_Separation_Training/models -o MDX23C-8KFFT-InstVoc_HQ.ckpt
  !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_2_stem_full_band_8k.yaml -d Music_Source_Separation_Training/models -o model_2_stem_full_band_8k.yaml
  if not precompiled:
    !rm Music_Source_Separation_Training/requirements.txt
    !wget -P Music_Source_Separation_Training "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/Music-Source-Separation/requirements.txt"
    !pip install requirements.txt

def install_project():
  !git clone https://github.com/ShiromiyaG/RVC-AI-Cover-Maker.git

def install_precompiled_dependencies():
  arquivo_zip = '/content/RVCAICoverMakerDeps.tar.gz'
  pasta_destino = '/content/dependencies'
  Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
  !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/RVCAICoverMakerDeps.tar.gz -d /content -o RVCAICoverMakerDeps.tar.gz
  source_path = "/content/RVCAICoverMakerDeps.tar.gz"
  destination_path = "/"
  extract_tar_file(source_path, destination_path)

def install_dependencies(precompiled=True):
  !sudo apt-get install ffmpeg python3.10-venv aria2 libsamplerate0-dev
  !pip install transformers==4.28.0
  if not precompiled:
    !pip install poetry pedalboard
    !poetry config virtualenvs.create false
    !pip install --no-deps pyrubberband dora-search retrying hydra-core>=1.1
    !pip install yt-dlp[default] wget git+https://github.com/IAHispano/gdown
    !pip install --upgrade --upgrade-strategy only-if-needed pydub soxr cmake audiofile samplerate==0.1.0
    !pip install python-dotenv samplerate==0.1.0

def setup_ultimatevocalremover_api():
  !git clone https://github.com/NextAudioGen/ultimatevocalremover_api.git
  !rm ultimatevocalremover_api/requirements.txt
  !rm ultimatevocalremover_api/src/models_dir/models.json
  !rm ultimatevocalremover_api/src/models_dir/mdx/modelparams/model_data.json
  !rm ultimatevocalremover_api/src/models_dir/vr_network/modelparams/model_data.json
  !wget -P ultimatevocalremover_api/src/models_dir/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/models.json"
  !wget -P ultimatevocalremover_api/src/models_dir/mdx/modelparams/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/mdx/model_data.json"
  !wget -P ultimatevocalremover_api/src/models_dir/vr_network/modelparams/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/vr/model_data.json"
  !wget -P ultimatevocalremover_api/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/requirements.txt"
  %cd ultimatevocalremover_api
  !pip install --upgrade --upgrade-strategy only-if-needed .
  %cd ../

def setup_kitware():
  !wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | gpg --dearmor - | sudo tee /etc/apt/trusted.gpg.d/kitware.gpg >/dev/null
  !sudo apt-add-repository "deb https://apt.kitware.com/ubuntu/ $(lsb_release -cs) main" -y
  !sudo apt update
  !sudo apt install kitware-archive-keyring
  !sudo rm /etc/apt/trusted.gpg.d/kitware.gpg
  !sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 6AF7F09730B3F0A4
  !sudo apt update
  !sudo apt-get install libsamplerate0-dev g++ make cmake

def setup_orpheusdl(precompiled=True,not_clean_outputs=False):
  !git clone https://github.com/OrfiTeam/OrpheusDL.git
  if not precompiled:
    %cd OrpheusDL
    if not_clean_outputs:
      !poetry install --no-root
    else:
      !poetry install --no-root -q > /dev/null 2>&1
    %cd ../
  !git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
  !wget -P OrpheusDL/config "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/OrpheusDL/settings.json"

def setup_rvc_cli(precompiled=True,not_clean_outputs=False):
  !git clone https://github.com/shirounanashi/RVC_CLI.git
  if not precompiled:
    %cd RVC_CLI
    if not_clean_outputs:
      !poetry install --no-root
    else:
      !poetry install --no-root > /dev/null 2>&1
    %cd ../
  !wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/fcpe.pt"
  !wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/hubert_base.pt"
  !wget -P ./RVC_CLI "https://huggingface.co/IAHispano/Applio/resolve/main/Resources/rmvpe.pt"

def main(precompiled=True,not_clean_outputs=False):
  install_project()
  os.chdir("/content/RVC-AI-Cover-Maker")
  if precompiled:
    install_precompiled_dependencies()
  else:
    setup_ultimatevocalremover_api()
  install_dependencies(precompiled)
  setup_kitware()
  setup_orpheusdl(precompiled,not_clean_outputs)
  setup_rvc_cli(precompiled,not_clean_outputs)
  !pip uninstall onnxruntime-gpu -y
  !python -m pip -q install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

main(precompiled, not_clean_outputs)

# **Send audio file**

In [ ]:
#@title ### **Local file upload**
from google.colab import files
import os
import fnmatch
from pathlib import Path
Path('/content/drive/MyDrive/RVCAICoverMakerInput').mkdir(parents=True, exist_ok=True)
def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".flac") or file.endswith(".wav"):
                caminho_arquivo = os.path.join(root, file)
                arquivos.append(caminho_arquivo)
    return arquivos
uploaded = files.upload()
uploaded = procurar_arquivos("/content")
for arquivo in uploaded:
  !cp "{arquivo}" "/content/drive/MyDrive/RVCAICoverMakerInput"

In [ ]:
#@title ### **Files from Google Drive**
import os
import fnmatch
from google.colab import drive
from pathlib import Path
import importlib
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
drive_path = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}

def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("audio-separator"):
  pass
else:
  print("Dependencies not installed, setting up the Drive now")
  drive.mount('/content/drive')

arquivos_encontrados = procurar_arquivos(drive_path)

for arquivo in arquivos_encontrados:
    !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title ### **Download file from YouTube or Deezer**
from pathlib import Path
import os
os.environ['LANGUAGE'] = 'en_US.UTF-8'
# @markdown #### Select one of the options to download a song
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ###### If you are going to use Deezer to download
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
input_folder = '/content/musicas/arquivos-originais'

if link_of_yt != "":
  !python inference.py download_yt --link "{link_of_yt}"

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    !python inference.py download_deezer --link "{link_of_deezer}" \
          --bf_secret "{bf_secret}" \
          --track_url_key "{track_url_key}" \
          --arl "{arl}"
  else:
    raise ValueError("You need to provide the bf secret, track url key arl of your Deezer Premium account")

# **Programs**

In [ ]:
#@title ### **Remove Backing Vocals and Echo/Reverb**
from pathlib import Path
import os
from glob import glob
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" # @param {type:"string"}

no_back_folder = "/content/musicas/sem-back"
output_vocals_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(no_back_folder).mkdir(parents=True, exist_ok=True)
Path(output_vocals_folder).mkdir(parents=True, exist_ok=True)
device = "cuda"

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_file), "files")
  !python inference.py remove_backing_vocals_and_reverb \
            --input_file "{input_file}" \
            --no_back_folder "{no_back_folder}" \
            --output_folder "{output_vocals_folder}" \
            --device "{device}"


In [ ]:
#@title ### **Remove Instrumentals, Backing Vocals and Echo/Reverb**
# @markdown ##### Leave the imput folder like this to use the songs downloaded from YT/Deezer
from pathlib import Path
import os
from glob import glob
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/musicas/arquivos-originais" # @param {type:"string"}
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_vocals_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(no_back_folder).mkdir(parents=True, exist_ok=True)
Path(no_inst_folder).mkdir(parents=True, exist_ok=True)
Path(output_vocals_folder).mkdir(parents=True, exist_ok=True)
# @markdown ##### Merge Spectrograms
vocals_ensemble = True #@param {type:"boolean"}
algorithm_ensemble_vocals =  "averege" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}
input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_file), "files")
  !python inference.py separate_vocals \
            --input_file "{input_file}" \
            --vocal_ensemble "{Vocals_ensemble}" \
            --algorithm_ensemble_vocals "{algorithm_ensemble_vocals}" \
            --no_inst_folder "{no_inst_folder}" \
            --no_back_folder "{no_back_folder}" \
            --output_folder "{output_vocals_folder}" \
            --device "{device}"


In [ ]:
#@title ### **Remove Vocals (Keeping Backing Vocals)**
#@markdown ##### Leave the imput folder like this to use the songs downloaded from YT/Deezer
from pathlib import Path
import os
from glob import glob
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/musicas/arquivos-originais" # @param {type:"string"}
stage1_dir = "/content/musicas/inst-etapa1"
stage2_dir = "/content/musicas/inst-etapa2"
output_drive = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(stage1_dir).mkdir(parents=True, exist_ok=True)
Path(stage2_dir).mkdir(parents=True, exist_ok=True)
Path(output_drive).mkdir(parents=True, exist_ok=True)

Instrumental_Ensemble = True
device = "cuda"

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_file), "files")
  !python inference.py separate_instrumentals \
            --input_file "{input_file}" \
            --Instrumental_Ensemble "{Instrumental_Ensemble}" \
            --algorithm_ensemble_inst "Max Spec" \
            --stage1_dir "{stage1_dir}" \
            --stage2_dir "{stage2_dir}" \
            --final_output_dir "{output_drive}" \
            --device "{device}" \


In [ ]:
#@title ### **Apply Reverb**
from pathlib import Path
import os
from glob import glob
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
REVERB_SIZE = 0.15  # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_WETNESS = 0.2 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_DRYNESS = 0.8 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
REVERB_DAMPING = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
output_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(output_folder).mkdir(parents=True, exist_ok=True)

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_file), "files")
  !python inference.py reverb \
            --audio_path "{input_file}" \
            --reverb_size "{REVERB_SIZE}" \
            --reverb_wetness "{REVERB_WETNESS}" \
            --reverb_dryness "{REVERB_DRYNESS}" \
            --reverb_damping "{REVERB_DAMPING}" \
            --output_format "flac" \
            --output_path "{output_folder}"

In [ ]:
#@title ### **Remove RVC noise**
from pathlib import Path
import os
from glob import glob
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
noise_db_limit = -30 # @param {type:"slider", min:-50, max:0, step:0.1}
output_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(output_folder).mkdir(parents=True, exist_ok=True)

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py remove_noise \
            --audio_path "{input_file}" \
            --noise_db_limit "{noise_db_limit}" \
            --output_path "{output_folder}"

In [ ]:
#@title ### **Ensemble**
from pathlib import Path
import os
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
output_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput" #@param {type:"string"}
Path(output_folder).mkdir(parents=True, exist_ok=True)
algorithm = "averege" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}
files = [file for file in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, file))]
common_name = os.path.commonprefix(files)

!python inference.py ensemble \
            --input_folder "{input_folder}" \
            --algorithm_ensemble "{algorithm}" \
            --output_path "{output_folder}/{common_name}.wav"


In [ ]:
#@title ### **RVC inference**
from pathlib import Path
import os
from glob import glob
os.environ['LANGUAGE'] = 'en_US.UTF-8'
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" # @param {type:"string"}
rvc_model_link = ""  # @param {type:"string"}
rvc_model_name = os.path.splitext(os.path.basename(rvc_model_link))
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
pitch = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
autotune = False  # @param{type:"boolean"}
output_drive = "/content/drive/MyDrive/RVCAICoverMakerOutput"
model_destination_folder = "/content/RVC_CLI/logs"

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py rvc_ai \
            --input_path "{input_file}" \
            --output_path "{output_drive}" \
            --rvc_model_name "{rvc_model_name}" \
            --model_destination_folder "{model_destination_folder}" \
            --rvc_model_link "{rvc_model_link}" \
            --pitch "{pitch}" \
            --filter_radius "{filter_radius}" \
            --index_rate "{index_rate}" \
            --hop_length "{hop_length}" \
            --rms_mix_rate "{rms_mix_rate}" \
            --protect "{protect}" \
            --autotune "{autotune}" \
            --f0method "{f0method}" \
            --split_audio "{split_audio}" \
            --clean_audio "{clean_audio}" \
            --clean_strength "{clean_strength}" \
            --export_format "FLAC"
